# Data preparation and pre-processing
This part would examine the data and perform data-wrangling and cleaning so to prepare the data for ML model training in the next section

In [1]:
import numpy as np
import pandas as pd
from datetime import datetime

# Preprocessing
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, MinMaxScaler, RobustScaler, StandardScaler
import category_encoders as ce

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from scipy import stats


In [2]:
plotting = False

In [3]:
df_singa_airbnb = pd.read_csv('listings.csv')
df_singa_airbnb.head()

,id,name,host_id,host_name,neighbourhood_group,neighbourhood,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,last_review,reviews_per_month,calculated_host_listings_count,availability_365
0,49091,COZICOMFORT LONG TERM STAY ROOM 2,266763,Francesca,North Region,Woodlands,1.44255,103.79580,Private room,83,180,1,2013-10-21,0.01,2,365
1,50646,Pleasant Room along Bukit Timah,227796,Sujatha,Central Region,Bukit Timah,1.33235,103.78521,Private room,81,90,18,2014-12-26,0.28,1,365
2,56334,COZICOMFORT,266763,Francesca,North Region,Woodlands,1.44246,103.79667,Private room,69,6,20,2015-10-01,0.20,2,365
3,71609,Ensuite Room (Room 1 & 2) near EXPO,367042,Belinda,East Region,Tampines,1.34541,103.95712,Private room,206,1,14,2019-08-11,0.15,9,353
4,71896,B&B Room 1 near Airport & EXPO,367042,Belinda,East Region,Tampines,1.34567,103.95963,Private room,94,1,22,2019-07-28,0.22,9,355


At a glance of the features given, the processs to prepare them ML models are summarised below

### Prediction column
- price

### Feature columns
**Columns to be removed**
- host_name
- name
- last_review
- id
- host_id (since calculated_host_listings_count is available, host_id does not seem to add much value)

**Categorical encoding**

*Categorical values*
(This is applicable when there is a ranking in the categorical number)
- room_type


*OneHot encoding*
(This is applicable when there is no ranking)
- neighbourhood
- neighbourhood_group


**Scaling**

*MinMaxScaler (for those features that do not seem to have outliers and more evenly distributed)*
- calculated_host_listings_count
- availability_365
- latitude
- longitude

*RobustScaler (For those features that seem to be skewed and have outliers)*
- minimum_nights
- number_of_reviews
- reviews_per_month


### 1. Check if there are any missing values and perform imputation for those missing values

In [4]:
df = df_singa_airbnb.copy()

In [5]:
# dict_mat_null = {}
# dict_df_col_null = {}

# for col in df.columns:
#     dict_mat_null[col] = pd.isnull(df[col])
#     if not df[dict_mat_null[col]].empty:
#         dict_df_col_null[col] = df[dict_mat_null[col]]

# dict_df_col_null

In [6]:
null_value_stats = df.isnull().sum()
null_value_stats[null_value_stats != 0]

name                    2
last_review          2758
reviews_per_month    2758
dtype: int64

#### Imputation
Because name and last_review are going to be removed, imputation for them is not necessary

For rows with Null reviews_per_month, they will be imputed with 0 assuming that no value means no comments ever made

In [7]:
df['reviews_per_month'] = df['reviews_per_month'].fillna(0)

### Removing unnecessary features

In [8]:
df = df.drop(['name', 'id', 'host_name', 'host_id', 'last_review'], axis=1)

### Outliers removal/cleaning
Plotting the distribution to check for outliers, then clean the outliers (either remove or impute to other values), and re-plot again

In [9]:
numeric_features = ['latitude','longitude','price',
                    'minimum_nights','number_of_reviews', 'reviews_per_month',
                    'calculated_host_listings_count','availability_365']

In [10]:
# Checking outlier using boxplot
if plotting:
    df_before = df_singa_airbnb.copy()

    plt.figure(figsize=(18,10))

    for i in range(1,9):    
        plt.subplot(2,4,i)
        fig = df_before.boxplot(column=numeric_features[i-1])

In [11]:
# Checking outlier using distribution
if plotting:
    plt.figure(figsize=(18,10))

    for j in range(1,9):
        plt.subplot(2,4,j)
        sns.histplot(df_before[numeric_features[j-1]])

From the boxplot and distribution above, it seems unreasonable to include some of the extreme values in price and minimum nights. 

Therefore, a z-score based algorithm is used to remove the outliers in price from training. 

As for minimum nights, it does not make sense to request for one-year minimum nights. So those rows with more than 356 minimum nights would be dropped off



In [12]:
stats.zscore(df['price'])

array([-0.2537966 , -0.25967608, -0.29495297, ..., -0.32729011,
       -0.33316959, -0.30671193])

In [13]:
# outliers = [] 

# # Check outliers using zscore
# for lat in df['latitude']:
#     zscore = (lat - np.mean(df['latitude'])) / np.std(df['latitude'])
#     if zscore > 3:
#         outliers.append(lat)
        
# print(len(outliers))

# df = df.replace(outliers, np.median(df['latitude']))

In [14]:
# Remove outlier from price (target)
# outlier = (np.abs(stats.zscore(df['price'])) < 1.6)
# outlier_ix = np.where(outlier==False)
# df.drop(index=outlier_ix[0], inplace=True)

# The price is heavily right-skewed and so removing high price values would make the model better
df = df[df['price'] < 500]

In [15]:
# Remove rows with minimum nights > 365
df = df[df['minimum_nights'] <= 365]

In [16]:
# Checking outlier after cleaning using boxplot
if plotting:
    plt.figure(figsize=(18,10))

    for i in range(1,9):    
        plt.subplot(2,4,i)
        fig = df.boxplot(column=numeric_features[i-1])

In [17]:
# Checking outlier after cleaning using distribution
if plotting:
    plt.figure(figsize=(18,10))

    for j in range(1,9):
        plt.subplot(2,4,j)
        sns.histplot(df[numeric_features[j-1]])

### 2. Categorical encoding

For room_type, there is intrinsic ranking in between shared room, private room and Entire home/apt. Therefore, they are assigned values 1, 2 and 3 respectively

In [18]:
# Map integers to categorical values
room_dict ={
    'Shared room': 1,
    'Private room': 2,
    'Entire home/apt': 3
}

df['room_type'] = df['room_type'].map(room_dict)

In [19]:
# binary = ce.BinaryEncoder(cols=['neighbourhood'])
# df = binary.fit_transform(df)

In [20]:
#neighbourhood_group
#neighbourhood
#room_type

# print(df['neighbourhood_group'].unique())
# print(df['neighbourhood'].unique())
# print(df['room_type'].unique())

# oh_enc = OneHotEncoder()
ord_enc = OrdinalEncoder()
col_encode = ['neighbourhood_group','neighbourhood']

# for col in col_encode:
#     df = oh_enc.fit_transform(np.array(df_singa_airbnb[col]).reshape(-1,1))

# df[col_encode] =  ord_enc.fit_transform(df[col_encode])

df = pd.get_dummies(df, columns=col_encode, prefix=col_encode)


In [21]:
df

,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,neighbourhood_group_Central Region,...,neighbourhood_Serangoon,neighbourhood_Singapore River,neighbourhood_Southern Islands,neighbourhood_Sungei Kadut,neighbourhood_Tampines,neighbourhood_Tanglin,neighbourhood_Toa Payoh,neighbourhood_Western Water Catchment,neighbourhood_Woodlands,neighbourhood_Yishun
0,1.44255,103.79580,2,83,180,1,0.01,2,365,0,...,0,0,0,0,0,0,0,0,1,0
1,1.33235,103.78521,2,81,90,18,0.28,1,365,1,...,0,0,0,0,0,0,0,0,0,0
2,1.44246,103.79667,2,69,6,20,0.20,2,365,0,...,0,0,0,0,0,0,0,0,1,0
3,1.34541,103.95712,2,206,1,14,0.15,9,353,0,...,0,0,0,0,1,0,0,0,0,0
4,1.34567,103.95963,2,94,1,22,0.22,9,355,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7901,1.27973,103.78751,3,100,3,0,0.00,31,61,1,...,0,0,0,0,0,0,0,0,0,0
7902,1.27973,103.78751,3,100,3,0,0.00,31,61,1,...,0,0,0,0,0,0,0,0,0,0
7904,1.31286,103.85996,2,58,30,0,0.00,3,173,1,...,0,0,0,0,0,0,0,0,0,0
7905,1.29543,103.83801,2,56,14,0,0.00,2,30,1,...,0,0,0,0,0,0,0,0,0,0


### 3. Scaling

**StandardScaler**

In [22]:
col_scale = ['calculated_host_listings_count', 'availability_365', 'latitude', 'longitude', 'minimum_nights', 'number_of_reviews', 'reviews_per_month']

s_scaler = StandardScaler()

df_scaled = df.copy()

for col in col_scale:
    df_scaled[col] = s_scaler.fit_transform(np.array(df_scaled[col]).reshape(-1,1))

**MinMaxScaler**

In [23]:
# col_mm_scale = ['calculated_host_listings_count', 'availability_365', 'latitude', 'longitude']

# mm_scaler = MinMaxScaler()

# df_scaled = df.copy()

# for col in col_mm_scale:
#     df_scaled[col] = mm_scaler.fit_transform(np.array(df_scaled[col]).reshape(-1,1))

In [24]:
# col_ro_scale = ['minimum_nights', 'number_of_reviews', 'reviews_per_month']

# ro_scaler = RobustScaler()

# for col in col_ro_scale:
#     df_scaled[col] = ro_scaler.fit_transform(np.array(df_scaled[col]).reshape(-1,1))

In [25]:
df.head()

,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,neighbourhood_group_Central Region,...,neighbourhood_Serangoon,neighbourhood_Singapore River,neighbourhood_Southern Islands,neighbourhood_Sungei Kadut,neighbourhood_Tampines,neighbourhood_Tanglin,neighbourhood_Toa Payoh,neighbourhood_Western Water Catchment,neighbourhood_Woodlands,neighbourhood_Yishun
0,1.44255,103.79580,2,83,180,1,0.01,2,365,0,...,0,0,0,0,0,0,0,0,1,0
1,1.33235,103.78521,2,81,90,18,0.28,1,365,1,...,0,0,0,0,0,0,0,0,0,0
2,1.44246,103.79667,2,69,6,20,0.20,2,365,0,...,0,0,0,0,0,0,0,0,1,0
3,1.34541,103.95712,2,206,1,14,0.15,9,353,0,...,0,0,0,0,1,0,0,0,0,0
4,1.34567,103.95963,2,94,1,22,0.22,9,355,0,...,0,0,0,0,1,0,0,0,0,0


In [26]:
df_scaled.head()

,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,neighbourhood_group_Central Region,...,neighbourhood_Serangoon,neighbourhood_Singapore River,neighbourhood_Southern Islands,neighbourhood_Sungei Kadut,neighbourhood_Tampines,neighbourhood_Tanglin,neighbourhood_Toa Payoh,neighbourhood_Western Water Catchment,neighbourhood_Woodlands,neighbourhood_Yishun
0,4.191170,-1.212842,2,83,4.172766,-0.401964,-0.587542,-0.592459,1.079569,0,...,0,0,0,0,0,0,0,0,1,0
1,0.587895,-1.454780,2,81,1.864938,0.163786,-0.354385,-0.607676,1.079569,1,...,0,0,0,0,0,0,0,0,0,0
2,4.188227,-1.192967,2,69,-0.289035,0.230345,-0.423469,-0.592459,1.079569,0,...,0,0,0,0,0,0,0,0,1,0
3,1.014926,2.472648,2,206,-0.417248,0.030668,-0.466646,-0.485938,0.997353,0,...,0,0,0,0,1,0,0,0,0,0
4,1.023427,2.529991,2,94,-0.417248,0.296904,-0.406198,-0.485938,1.011056,0,...,0,0,0,0,1,0,0,0,0,0


### 4. Final wrap up and saving processed data for ML training

In [27]:
df.to_pickle('./processed_data.p')

In [28]:
df_scaled.to_pickle('./processed_data_scaled.p')

In [29]:
df

,latitude,longitude,room_type,price,minimum_nights,number_of_reviews,reviews_per_month,calculated_host_listings_count,availability_365,neighbourhood_group_Central Region,...,neighbourhood_Serangoon,neighbourhood_Singapore River,neighbourhood_Southern Islands,neighbourhood_Sungei Kadut,neighbourhood_Tampines,neighbourhood_Tanglin,neighbourhood_Toa Payoh,neighbourhood_Western Water Catchment,neighbourhood_Woodlands,neighbourhood_Yishun
0,1.44255,103.79580,2,83,180,1,0.01,2,365,0,...,0,0,0,0,0,0,0,0,1,0
1,1.33235,103.78521,2,81,90,18,0.28,1,365,1,...,0,0,0,0,0,0,0,0,0,0
2,1.44246,103.79667,2,69,6,20,0.20,2,365,0,...,0,0,0,0,0,0,0,0,1,0
3,1.34541,103.95712,2,206,1,14,0.15,9,353,0,...,0,0,0,0,1,0,0,0,0,0
4,1.34567,103.95963,2,94,1,22,0.22,9,355,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7901,1.27973,103.78751,3,100,3,0,0.00,31,61,1,...,0,0,0,0,0,0,0,0,0,0
7902,1.27973,103.78751,3,100,3,0,0.00,31,61,1,...,0,0,0,0,0,0,0,0,0,0
7904,1.31286,103.85996,2,58,30,0,0.00,3,173,1,...,0,0,0,0,0,0,0,0,0,0
7905,1.29543,103.83801,2,56,14,0,0.00,2,30,1,...,0,0,0,0,0,0,0,0,0,0
